## Huspris

Her har vi data som beskriver boligpriser i Ames, Iowa fra 2006 til 2010. Datasettet inneholder mange variabler som kan brukes til å vurdere boligverdi. Se [her](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview) for mer informasjon om data. I denne oppgaven ser vi kun på de numeriske data. 

In [ ]:
# imports
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# les inn husprisdata
df = pd.read_csv('data/huspris.csv')

In [ ]:
# del data i mål-, og prediktorvariabler
reduced_df = df.select_dtypes(include=[np.number]).drop('Id', axis=1).dropna()
X = reduced_df.drop('SalePrice', axis=1).values
y = reduced_df['SalePrice'].values

Del data i trenings-, validerigns-, og testdata med størrelser 70%, 15%, 15% av data. 

In [ ]:
# dele data i trenings, validerings og testdata
# generer X_train, X_val, X_test, y_train, y_val, y_test
___

Tren en Grunnlinjemodell på treningsdata og finn validerings-RMSE. 

In [ ]:
baseline = ___
rmse_baseline = ___
rmse_baseline

Tren en Lasso regresjonsmodell (sklearn.linear_model.Lasso) med hyperparameter alpha mellom 1 og 500 på treningsdata. Sorter de ulike modellene etter mean kvadrert feil på valideringsdata (sklearn.metrics.mean_squared_error).
Visualiser hvordan mean kvadratisk feil avhenger av alpha. 

In [ ]:
# tren forskjellige modeller
lasso_models = {alpha: ___ for alpha in np.arange(1, 500, 10)}

for _, model in lasso_models.items():
    ___

In [ ]:
# sjekk MSE for valideringsdata
mse = ___

In [ ]:
# visualiser validerings-MSE avhengig av alpha
fig = ___
fig.show()

Lag alle polynomkombinasjoner av grad 2 av data (sklearn.preprocessing.PolynomialFeatures). 
Tren en Lasso regresjonsmodell (sklearn.linear_model.Lasso) med hyperparameter alpha mellom 500 og 1500 på polynomkombinasjoner av treningsdata.
Sorter de ulike modellene etter mean kvadrert feil på valideringsdata (sklearn.metrics.mean_squared_error).
Visualiser hvordan mean kvadratisk feil avhenger av alpha. 

In [ ]:
# lag datasett med polynomielle data
poly = ___
X_train_pf = poly.fit_transform(X_train)
X_val_pf = poly.transform(X_val)
X_test_pf = poly.transform(X_test)

In [ ]:
# tren forskjellige modeller
pf_models = ___



In [ ]:
# sjekk MSE for valideringsdata
pf_mse = ___

In [ ]:
# visualiser validerings-MSE avhengig av alpha
fig = ___
fig.show()

Se om du kan finne en bedre modell med en annen metode enn Lasso regresjon (e.g. sklearn.linear_model.ElasticNet, sklearn.ensemble.RandomForestRegressor, sklearn.svm.SVR, sklearn.gaussian_process.GaussianProcessRegressor). Finn gode hyperparametre til metoden du velger ut. 

In [ ]:
# tren forskjellige modeller
models = ___


In [ ]:
# sjekk MSE for valideringsdata
mse = ___

Velg ut den beste modellen og sjekk hvor godt den generaliserer ved å regne ut mean kvadrert feil og kvadratrooten av mean kvadrert feil på testdata. 

In [ ]:
# sjekk generaliseringsevne
best_model = ___
test_mse = ___

Gi en oppsummering over hva du har gjort og hva resultatet var. 

...